In [1]:
from keras.models import load_model
from keras_preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2 as cv

Using TensorFlow backend.


In [2]:
#加载模型

model1 = load_model("weights/densenet_0023.h5")
model2 = load_model("weights/inceptionv3_0016.h5")
model3 = load_model("weights/mobilenetv2_0032.h5")
model4 = load_model("weights/nasnet_0017.h5")
model5 = load_model("weights/vgg19two_0027.h5")

print("模型加载成功")

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


模型加载成功


In [17]:
#导入数据
def load_data(shape):
    test_path = '测试数据/测试7'
    test_batches = ImageDataGenerator(rescale=1/255).flow_from_directory(test_path,
                                      target_size=shape,
                                    classes=["C2F","X2F"],
                                    class_mode="binary",batch_size=10,shuffle=False)
    return test_batches

# 预测
def pred(model,steps,shape):
    steps = steps
    test_batches=load_data(shape)
    pred = model.predict_generator(test_batches, steps=steps, verbose=1)
    pred = pred.ravel()
    pred = list(pred)
    for i in range(len(pred)):
        if pred[i] < 0.5:
            pred[i] = 0
        else:
            pred[i] = 1
    return pred
#投票选出最多的
def vote(lt):
	index1 = 0
	max = 0
	for i in range(len(lt)):
		flag = 0
		for j in range(i+1,len(lt)):
			if lt[j] == lt[i]:
				flag += 1
		if flag > max:
			max = flag
			index1 = i
	return index1
def Ensemble(steps):
    ans = []
    pred1=list(pred(model1,steps,(224,224)))
    pred2=list(pred(model2,steps,(299,299)))
    pred3=list(pred(model3,steps,(224,224)))
    pred4=list(pred(model4,steps,(224,224)))
    pred5=list(pred(model5,steps,(224,224)))
    for i in range(len(pred5)):
        ls = []
        ls.append(pred1[i])
        ls.append(pred2[i])
        ls.append(pred3[i])
        ls.append(pred4[i])
        ls.append(pred5[i])
        ans.append(ls[vote(ls)])
    return ans

steps=3
#投票得出最终结果
predicts=Ensemble(steps)
# for i in enumerate(predicts):
#     print(i)



test_batches = load_data((224,224))
test_class = np.array([])

files=[]
for i in range(steps):
    test_imgs, test_lables = next(test_batches)
    test_class = np.hstack((test_class, test_lables))
    files.append(test_imgs)
# print("真实类别：", test_class)
# print("预测类别：", predicts)


# 打印混淆矩阵
cm = confusion_matrix(test_class, predicts)

print(cm)

tmp = 0
for i in range(len(cm[0, :])):
    tmp += cm[i][i]
accuracy = tmp / np.sum(cm)
print("acc:", accuracy)


# i=0
# for images in files:
#     for pred_label, image in zip(predicts, images):
#         i += 1
#         cv.imshow('{} - {} '.format(
#             i, pred_label), image)
# cv.waitKey(0)
# cv.destroyAllWindows()

Found 30 images belonging to 2 classes.


1/3 [=========>....................] - ETA: 0s

2/3 [===================>..........] - ETA: 0s

3/3 [==============================] - 0s 137ms/step


Found 30 images belonging to 2 classes.


...

1/3 [=========>....................] - ETA: 0s

2/3 [===================>..........] - ETA: 0s

3/3 [==============================] - 0s 131ms/step


Found 30 images belonging to 2 classes.


[[14  1]
 [ 6  9]]
acc: 0.7666666666666667
